# Predicting Heart Disease

This dataset contains 76 features, but all published experiments refer to using a subset of 14 of them. The "goal" feature refers to the presence of heart disease in the patient. It is integer valued from 0 (no presence) to 4 (values 1,2,3,4) from absence (value 0). It is therefore a multiclass classification problem.

*For our example, we will use several more features than the traditional 14.*

Feature info (attributes used): 
1. feature 3 (age) - Age in years
2. feature 4 (sex) - male or female
3. feature 9 (cp) - chest pain type (typical angina, atypical angina, non-anginal, asymptomatic)
4. feature 10 (trestbps) - resting blood pressure (mm Hg)
5. feature 12 (chol) - cholesterol (mg/dl)
6. feature 14 (cigperday) - number of cigarettes per day
7. feature 16 (fbs) - fasting blood sugar > 120 mg/dl (1 = true; 0 = false) 
8. feature 18 (famhist) - family history of heart disease (1 = true; 0 = false)
9. feature 19 (restecg) - resting electrocardiographic results (normal; st-t = having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV); vent = showing probable or definite left ventricular hypertrophy by Estes' criteria)
10. feature 32 (thalach) - maximum heart rate achieved
11. feature 38 (exang) - exercise induced angina (1 = yes; 0 = no)
12. feature 40 (oldpeak) - ST depression induced by exercise relative to rest
13. feature 41 (slope) - the slope of the peak exercise ST segment (upsloping, flat, downsloping)
14. feature 44 (ca) - number of major vessels (0-3) colored by flourosopy
15. feature 51 (thal) - normal, fixed defect, or reversable defect
16. feature 58 (target) (the predicted attribute) 
  - 0: < 50% diameter narrowing
  - 1+: > 50% diameter narrowing

### Our focus in using this dataset will be exploring pre-processing methods more thoroughly

More details can be found at [the UCI repository](https://archive.ics.uci.edu/ml/datasets/Heart+Disease).

### Acknowledgments

The authors of the dataset have requested that any use of the data include the names of the principal investigator responsible for the data collection at each institution. They would be: 

1. Hungarian Institute of Cardiology. Budapest: Andras Janosi, M.D. 
2. University Hospital, Zurich, Switzerland: William Steinbrunn, M.D. 
3. University Hospital, Basel, Switzerland: Matthias Pfisterer, M.D. 
4. V.A. Medical Center, Long Beach and Cleveland Clinic Foundation:Robert Detrano, M.D., Ph.D.

## Loading the data from CSV

We can read the data directly from the CSV located in the [data/](data/) directory. The [raw data](data/heart-disease-raw.csv) was pre-processed to re-name categorical features where they are otherwise ordinal variables. This allows us to walk through an entire pre-processing pipeline

In [1]:
import pandas as pd
import numpy as np

# read the raw csv
X = pd.read_csv('data/heart-disease-2.csv', header=None)

# rename the columns
cols = ['age', 'sex', 'cp', 'trestbps', 'chol', 'cigperday', 'fbs', 'famhist',
        'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

X.columns = cols
y = X.pop('target')  # don't want target in the X matrix
X.head()

,age,sex,cp,trestbps,chol,cigperday,fbs,famhist,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,male,typical anginal,145,233,50.0,1,1,vent,150,0,2.3,downsloping,0.0,fixed
1,67,male,asymptomatic,160,286,40.0,0,1,vent,108,1,1.5,flat,3.0,normal
2,67,male,asymptomatic,120,229,20.0,0,1,vent,129,1,2.6,flat,2.0,reversable
3,37,male,non-anginal,130,250,0.0,0,1,normal,187,0,3.5,downsloping,0.0,normal
4,41,female,atypical anginal,130,204,0.0,0,1,vent,172,0,1.4,upsloping,0.0,normal


## Pre-split: any major imbalance?

If there are any categorical features with rare factor levels that need to be considered before splitting, we'll find out here.

In [2]:
def examine_cats(frame):
    for catcol in frame.columns[frame.dtypes == 'object'].tolist():
        print(catcol)
        print(frame[catcol].value_counts())
        print("")
        
examine_cats(X)

sex
male      206
female     97
Name: sex, dtype: int64

cp
asymptomatic        144
non-anginal          86
atypical anginal     50
typical anginal      23
Name: cp, dtype: int64

restecg
normal    151
vent      148
st-t        4
Name: restecg, dtype: int64

slope
upsloping      142
flat           140
downsloping     21
Name: slope, dtype: int64

thal
normal        166
reversable    117
fixed          18
Name: thal, dtype: int64



# Perform train/test split

Remember, we always need to split! We will also stratify on the '`restecg`' variable since it's the most likely to be poorly split.

In [3]:
from sklearn.model_selection import train_test_split

seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed, 
                                                    stratify=X['restecg'])

print("Train size: %i" % X_train.shape[0])
print("Test size: %i" % X_test.shape[0])
X_train.head()

Train size: 227
Test size: 76


,age,sex,cp,trestbps,chol,cigperday,fbs,famhist,restecg,thalach,exang,oldpeak,slope,ca,thal
167,54,female,atypical anginal,132,288,0.0,1,0,vent,159,1,0.0,upsloping,1.0,normal
166,52,male,non-anginal,138,223,50.0,0,1,normal,169,0,0.0,upsloping,NaN,normal
300,57,male,asymptomatic,130,131,50.0,0,0,normal,115,1,1.2,flat,1.0,reversable
185,63,female,atypical anginal,140,195,2.0,0,1,normal,179,0,0.0,upsloping,2.0,normal
84,52,male,atypical anginal,120,325,30.0,0,1,normal,172,0,0.2,upsloping,0.0,normal


In [4]:
examine_cats(X_train)

sex
male      153
female     74
Name: sex, dtype: int64

cp
asymptomatic        105
non-anginal          66
atypical anginal     37
typical anginal      19
Name: cp, dtype: int64

restecg
normal    113
vent      111
st-t        3
Name: restecg, dtype: int64

slope
flat           110
upsloping      102
downsloping     15
Name: slope, dtype: int64

thal
normal        121
reversable     92
fixed          12
Name: thal, dtype: int64



# Custom Transformers

There are several custom transformers that will be useful for this data:

- Custom one-hot encoding that drops one level to avoid the [dummy variable trap](http://www.algosome.com/articles/dummy-variable-trap-regression.html)
- Model-based imputation of continuous variables, since mean/median centering is rudimentary

### Custom base class

We'll start with a cusom base class that depends on the input to be a Pandas dataframe. This base class will provide super methods for validating the input type as well as the presence of any prescribed columns.

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class CustomPandasTransformer(BaseEstimator, TransformerMixin):
    def _validate_input(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("X must be a DataFrame, but got type=%s" 
                            % type(X))
        return X
    
    @staticmethod
    def _validate_columns(X, cols):
        scols = set(X.columns)  # set for O(1) lookup
        if not all(c in scols for c in cols):
            raise ValueError("all columns must be present in X")

# One-hot encode categorical data

It is probably (hopefully) obvious why we need to handle data that is in string format. There is not much we can do numerically with data that resembles the following:

    [flat, upsloping, downsloping, ..., flat, flat, downsloping]
    
There is a natural procedure to force numericism amongst string data: map each unique string to a unique level (1, 2, 3). This is, in fact, exactly what the sklearn [`LabelEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) does. However, this is not sufficient for modeling purposes, since most algorithms will treat this as [ordinal data](https://en.wikipedia.org/wiki/Ordinal_data), where in many cases it is not. Imagine you fit a regression on data you've label-encoded, and one feature (type of chest pain, for instance) is now:

    [0, 2, 3, ..., 1, 0]
    
You might get coefficients back that make no sense since "asymptomatic" or "non-anginal", etc., are not inherently numerically greater or less than one another. Therefore, we [*one-hot encode*](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) our categorical data into a numerical representation. Now we have dummy data and a binary feature for each variable/factor-level combination.

In [5]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

class DummyEncoder(CustomPandasTransformer):
    def __init__(self, columns, sep='_', drop_one_level=True, tmp_nan_rep='N/A'):
        self.columns = columns
        self.sep = sep
        self.drop_one_level = drop_one_level
        self.tmp_nan_rep = tmp_nan_rep
        
    def fit(self, X, y=None):
        # validate the input
        X = self._validate_input(X).copy()  # get a copy
        
        # parameter validation happens here:
        tmp_nan = self.tmp_nan_rep
        
        # validate all the columns present
        cols = self.columns
        self._validate_columns(X, cols)
                
        # for each column, fit a label encoder
        lab_encoders = {}
        for col in cols:
            vec = [tmp_nan if pd.isnull(v) 
                   else v for v in X[col].tolist()]
            
            # if the tmp_nan value is not present in vec, make sure it is
            # so the transform won't break down
            svec = list(set(vec))
            if tmp_nan not in svec:
                svec.append(tmp_nan)
            
            le = LabelEncoder()
            lab_encoders[col] = le.fit(svec)
            
            # transform the column, re-assign
            X[col] = le.transform(vec)
            
        # fit a single OHE on the transformed columns - but we need to ensure
        # the N/A tmp_nan vals make it into the OHE or it will break down later.
        # this is a hack - add a row of all transformed nan levels
        ohe_set = X[cols]
        ohe_nan_row = {c: lab_encoders[c].transform([tmp_nan])[0] for c in cols}
        ohe_set = ohe_set.append(ohe_nan_row, ignore_index=True)
        ohe = OneHotEncoder(sparse=False).fit(ohe_set)
        
        # assign fit params
        self.ohe_ = ohe
        self.le_ = lab_encoders
        self.cols_ = cols
        
        return self
    
    def transform(self, X):
        check_is_fitted(self, 'ohe_')
        X = self._validate_input(X).copy()
        
        # fit params that we need
        ohe = self.ohe_
        lenc = self.le_
        cols = self.cols_
        tmp_nan = self.tmp_nan_rep
        sep = self.sep
        drop = self.drop_one_level
        
        # validate the cols and the new X
        self._validate_columns(X, cols)
        col_order = []
        drops = []
        
        for col in cols:
            # get the vec from X, transform its nans if present
            vec = [tmp_nan if pd.isnull(v) 
                   else v for v in X[col].tolist()]
            
            le = lenc[col]
            vec_trans = le.transform(vec)  # str -> int
            X[col] = vec_trans
            
            # get the column names (levels) so we can predict the 
            # order of the output cols
            classes = ["%s%s%s" % (col, sep, clz) for clz in le.classes_.tolist()]
            col_order.extend(classes)
            
            # if we want to drop one, just drop the last
            if drop:
                drops.append(classes[-1])
                
        # now we can get the transformed OHE
        ohe_trans = pd.DataFrame.from_records(data=ohe.transform(X[cols]), 
                                              columns=col_order)
        
        # set the index to be equal to X's for a smooth concat
        ohe_trans.index = X.index
        
        # if we're dropping one level, do so now
        if drops:
            ohe_trans = ohe_trans.drop(drops, axis=1)
        
        # drop the original columns from X
        X = X.drop(cols, axis=1)
        
        # concat the new columns
        X = pd.concat([X, ohe_trans], axis=1)
        return X

In [6]:
de = DummyEncoder(columns=['sex', 'cp', 'restecg', 'slope', 'thal'])
de.fit(X_train)
X_train_dummied = de.transform(X_train)
X_train_dummied.head()

,age,trestbps,chol,cigperday,fbs,famhist,thalach,exang,oldpeak,ca,...,cp_non-anginal,restecg_N/A,restecg_normal,restecg_st-t,slope_N/A,slope_downsloping,slope_flat,thal_N/A,thal_fixed,thal_normal
167,54,132,288,0.0,1,0,159,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
166,52,138,223,50.0,0,1,169,0,0.0,NaN,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
300,57,130,131,50.0,0,0,115,1,1.2,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
185,63,140,195,2.0,0,1,179,0,0.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
84,52,120,325,30.0,0,1,172,0,0.2,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Model-based imputation

As discussed in the iris notebook, there are many pitfalls to using mean or median for scaling. In instances where our data is too large to examine all features graphically, many times we cannot discern whether all features are normally distributed (a pre-requisite for mean-scaling). If we want to get more sophisticated, we can use an approach for imputation that is based on a model; we will use a [`BaggingRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) (since we are filling in NaN continuous variables only at this point).

Note that there are other common approaches for this, like KNN imputation, but nearest neighbors models require your data to be scaled, which we're trying to avoid.

### Beware:

Sometimes missing data is informative. For instance... failure to report `cigperday` could be a bias on part of the patient who may not want to receive judgment or a lecture, or could indicate 0.

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.externals import six

class BaggedRegressorImputer(CustomPandasTransformer):
    def __init__(self, impute_cols, base_estimator=None, n_estimators=10, 
                 max_samples=1.0, max_features=1.0, bootstrap=True, 
                 bootstrap_features=False, n_jobs=1,
                 random_state=None, verbose=0):
        
        self.impute_cols = impute_cols
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.bootstrap_features = bootstrap_features
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbose = verbose
        
    def fit(self, X, y=None):
        X = self._validate_input(X)  # don't need a copy this time
        
        # validate the columns
        cols = self.impute_cols
        self._validate_columns(X, cols)
        
        # drop off the columns we'll be imputing as targets
        regressors = {}
        targets = {c: X[c] for c in cols}
        X = X.drop(cols, axis=1)  # these should all be filled in (no NaN)
        
        for k, target in six.iteritems(targets):
            # split X row-wise into train/test where test is the missing
            # rows in the target
            test_mask = pd.isnull(target)
            train = X.loc[~test_mask]
            train_y = target[~test_mask]
            
            # fit the regressor
            regressors[k] = BaggingRegressor(
                base_estimator=self.base_estimator,
                n_estimators=self.n_estimators,
                max_samples=self.max_samples,
                max_features=self.max_features,
                bootstrap=self.bootstrap,
                bootstrap_features=self.bootstrap_features,
                n_jobs=self.n_jobs, 
                random_state=self.random_state,
                verbose=self.verbose, oob_score=False,
                warm_start=False).fit(train, train_y)
            
        # assign fit params
        self.regressors_ = regressors
        return self
        
    def transform(self, X):
        check_is_fitted(self, 'regressors_')
        X = self._validate_input(X).copy()  # need a copy
        
        cols = self.impute_cols
        self._validate_columns(X, cols)
        
        # fill in the missing
        models = self.regressors_
        for k, model in six.iteritems(models):
            target = X[k]
            
            # split X row-wise into train/test where test is the missing
            # rows in the target
            test_mask = pd.isnull(target)
            
            # if there's nothing missing in the test set for this feature, skip
            if test_mask.sum() == 0:
                continue
            test = X.loc[test_mask].drop(cols, axis=1)  # drop impute cols
            
            # generate predictions
            preds = model.predict(test)
            
            # impute!
            X.loc[test_mask, k] = preds
            
        return X

In [ ]:
bagged_imputer = BaggedRegressorImputer(impute_cols=['cigperday', 'ca'], 
                                        random_state=seed)
bagged_imputer.fit(X_train_dummied)

# save the masks so we can look at them afterwards
ca_nan_mask = pd.isnull(X_train_dummied.ca)
cpd_nan_mask = pd.isnull(X_train_dummied.cigperday)

# impute
X_train_imputed = bagged_imputer.transform(X_train_dummied)
X_train_imputed.head()

In [ ]:
X_train_imputed[ca_nan_mask].ca

In [ ]:
X_train_imputed[cpd_nan_mask].cigperday

# Feature selection/dimensionality reduction

Often times, when there is very high-dimensional data (100s or 1000s of features), it's useful to perform feature selection techniques to create more simple models that can be understood by analysts. A common one is [principal components analysis](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), but one of its drawbacks is diminished model clarity.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train_imputed)

# fit PCA, get explained variance of ALL features
pca_all = PCA(n_components=None)
pca_all.fit(scaler.transform(X_train_imputed))

In [ ]:
explained_var = np.cumsum(pca_all.explained_variance_ratio_)
explained_var

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

x_axis = np.arange(X_train_imputed.shape[1]) + 1
plt.plot(x_axis, explained_var)

# At which point to cut off?
minexp = np.where(explained_var > 0.9)[0][0]
plt.axvline(x=minexp, linestyle='dashed', color='red', alpha=0.5)
plt.xticks(x_axis)
plt.show()

print("Cumulative explained variance at %i components: %.5f" % (minexp, explained_var[minexp]))

At 15 (of 25) features, we finally explain >90% cumulative variance in our components. This is not a significant enough feature reduction to warrant use of PCA, so we'll skip it.

# Baseline several models

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

# set up our CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
stages = [
    ('dummy', DummyEncoder(columns=['sex', 'cp', 'restecg', 'slope', 'thal'])),
    ('impute', BaggedRegressorImputer(impute_cols=['cigperday', 'ca'], random_state=seed))
]

def build_pipeline(pipe_stages, estimator, est_name='clf'):
    # copy the stages
    pipe_stages = [stage for stage in pipe_stages]
    pipe_stages.append((est_name, estimator))
    
    # return the pipe
    return Pipeline(pipe_stages)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# fit a Logistic regression
lgr_pipe = build_pipeline(stages, LogisticRegression(random_state=seed))
cross_val_score(lgr_pipe, X=X_train, y=y_train, scoring='neg_log_loss', cv=cv)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# fit a GBM
gbm_pipe = build_pipeline(stages, GradientBoostingClassifier(n_estimators=25, max_depth=3, random_state=seed))
cross_val_score(gbm_pipe, X=X_train, y=y_train, scoring='neg_log_loss', cv=cv)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# fit a RF
rf_pipe = build_pipeline(stages, RandomForestClassifier(n_estimators=25, random_state=seed))
cross_val_score(rf_pipe, X=X_train, y=y_train, scoring='neg_log_loss', cv=cv)

# Tuning hyper-params

Now that we've baselined several models, let's choose a couple of the better-performing models to tune.

In [ ]:
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

gbm_pipe = Pipeline([
    ('dummy', DummyEncoder(columns=['sex', 'cp', 'restecg', 'slope', 'thal'])),
    ('impute', BaggedRegressorImputer(impute_cols=['cigperday', 'ca'], random_state=seed)),
    ('clf', GradientBoostingClassifier(random_state=seed))
])

# define the hyper-params
hyper_params = {
    'impute__n_estimators': randint(10, 50),
    'impute__max_samples': uniform(0.75, 0.125),
    'impute__max_features': uniform(0.75, 0.125),
    'clf__n_estimators': randint(50, 400),
    'clf__max_depth': [1, 3, 4, 5, 7],
    'clf__learning_rate': uniform(0.05, 0.1),
    'clf__min_samples_split': [2, 4, 5, 10],
    'clf__min_samples_leaf': [1, 2, 5]
}

# define the search
gbm_search = RandomizedSearchCV(gbm_pipe, param_distributions=hyper_params,
                                random_state=seed, cv=cv, n_iter=100,
                                n_jobs=-1, verbose=1, scoring='neg_log_loss',
                                return_train_score=False)

gbm_search.fit(X_train, y_train)

In [ ]:
lgr_pipe = Pipeline([
    ('dummy', DummyEncoder(columns=['sex', 'cp', 'restecg', 'slope', 'thal'])),
    ('impute', BaggedRegressorImputer(impute_cols=['cigperday', 'ca'], random_state=seed)),
    ('clf', LogisticRegression(random_state=seed))
])

# define the hyper-params
hyper_params = {
    'impute__n_estimators': randint(10, 50),
    'impute__max_samples': uniform(0.75, 0.125),
    'impute__max_features': uniform(0.75, 0.125),
    'clf__penalty': ['l1', 'l2'],
    'clf__C': uniform(0.5, 0.125),
    'clf__max_iter': randint(100, 500)
}

# define the search
lgr_search = RandomizedSearchCV(lgr_pipe, param_distributions=hyper_params,
                                random_state=seed, cv=cv, n_iter=100,
                                n_jobs=-1, verbose=1, scoring='neg_log_loss',
                                return_train_score=False)

lgr_search.fit(X_train, y_train)

# Examine the results

Right away we can tell that the logistic regression model was *much* faster than the gradient boosting model. However, does the extra time spent fitting end up giving us a performance boost? Let's introduce our test set to the optimized models and select the one that performs better. We are using [__log loss__](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html) as a scoring metric.

See [this answer](https://stats.stackexchange.com/questions/208443/intuitive-explanation-of-logloss) for a full intuitive explanation of log loss, but note that lower (closer to zero) is better. There is no maximum to log loss, and typically, the more classes you have, the higher it will be.

In [ ]:
from sklearn.metrics import log_loss

gbm_preds = gbm_search.predict_proba(X_test)
lgr_preds = lgr_search.predict_proba(X_test)

print("GBM test LOSS: %.5f" % log_loss(y_true=y_test, y_pred=gbm_preds))
print("Logistic regression test LOSS: %.5f" % log_loss(y_true=y_test, y_pred=lgr_preds))

Note that in log loss, greater is WORSE. Therefore, the logistic regression was out-performed by the GBM. If the greater time to fit is not an issue for you, then this would be the better model to select. Likewise, you may favor model transparency over the extra few decimal points of accuracy, in which case the logistic regression might be favorable.

# Variable importance

Most times, it's not enough to build a good model. Most executives will want to know *why* something works. Moreover, in regulated industries like banking or insurance, knowing why a model is working is incredibly important for defending models to a regulatory board.

One of the methods commonly used for observing variable importance for non-linear methods (like our gradient boosting model) is to break the model into piecewise linear functions and measure how the model performs against each variable. This is called a "partial dependency plot."

### Raw feature importances

We can get the raw feature importances from the estimator itself, and match them up with the transformed column names:

In [ ]:
# feed data through the pipe stages to get the transformed feature names
X_trans = X_train
for step in gbm_search.best_estimator_.steps[:-1]:
    X_trans = step[1].transform(X_trans)
    
transformed_feature_names = X_trans.columns
transformed_feature_names

In [ ]:
best_gbm = gbm_search.best_estimator_.steps[-1][1]
importances = best_gbm.feature_importances_
importances

In [ ]:
feature_importances = sorted(zip(np.arange(len(transformed_feature_names)), 
                                 transformed_feature_names, 
                                 importances), 
                             key=(lambda ici: ici[2]),
                             reverse=True)

feature_importances

### Partial dependency

In the following section, we'll break our GBM into a piecewise linear functions to gauge how different variables impact the target, and create [partial dependency plots](http://scikit-learn.org/stable/auto_examples/ensemble/plot_partial_dependence.html)

In [ ]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence

def plot_partial(est, which_features, X, names, label):
    fig, axs = plot_partial_dependence(est, X, which_features,
                                       feature_names=names,
                                       n_jobs=3, grid_resolution=50,
                                       label=label)
    
    fig.suptitle('Partial dependence of %i features\n'
                 'on heart disease' % (len(which_features)))
    plt.subplots_adjust(top=0.8)  # tight_layout causes overlap with suptitle
        
plot_partial(est=best_gbm, X=X_trans,
             which_features=[2, 8, 9, 0, 6, (2, 9)],
             names=transformed_feature_names,
             label=1)